<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

11928


In [2]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain'
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'mouse_genome_1000')

In [4]:
# laod encoding
# summarize total readout usage
encoding_folder = os.path.join(pool_folder, f'Encoding_design')
region_2_bits = pickle.load(open(os.path.join(encoding_folder, 'region_2_bits.pkl'), 'rb'))

In [61]:
# load probes
probes = []
final_pb_filename = os.path.join(pool_folder, 'final_pool_probes.fasta')

with open(final_pb_filename, 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        probes.append(record)


In [23]:
# load region names

region_2_names = {}

for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    #print(_reg,_gene)
    
    if _gene not in region_2_names:
        region_2_names[_gene] = _reg
print(region_2_names)

{'1': '1:3740000-3760000', '2': '1:6240000-6260000', '3': '1:8740000-8760000', '4': '1:11240000-11260000', '5': '1:13740000-13760000', '6': '1:16240000-16260000', '7': '1:18740000-18760000', '8': '1:21240000-21260000', '9': '1:23740000-23760000', '10': '1:26240000-26260000', '11': '1:28740000-28760000', '12': '1:31240000-31260000', '13': '1:33740000-33760000', '14': '1:36240000-36260000', '15': '1:38740000-38760000', '16': '1:41240000-41260000', '17': '1:43740000-43760000', '18': '1:46240000-46260000', '19': '1:48740000-48760000', '20': '1:51240000-51260000', '21': '1:53740000-53760000', '22': '1:56240000-56260000', '23': '1:58740000-58760000', '24': '1:61240000-61260000', '25': '1:63740000-63760000', '26': '1:66240000-66260000', '27': '1:68740000-68760000', '28': '1:71240000-71260000', '29': '1:73740000-73760000', '30': '1:76240000-76260000', '31': '1:78740000-78760000', '32': '1:81240000-81260000', '33': '1:83740000-83760000', '34': '1:86240000-86260000', '35': '1:88740000-88760000',

In [8]:
# laod encoding
# summarize total readout usage
region_2_readout_dict = pickle.load(open(os.path.join(library_folder, 'region_2_readout.pkl'), 'rb'))

In [10]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))

In [11]:
readout_dict

{'c': [SeqRecord(seq=Seq('AACCACGGGCTAGAAATATCCGAGTGGCAT'), id='NDB_784', name='NDB_784', description='NDB_784', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCTAAGACGTGCGACATGTTATGCTAGGC'), id='NDB_755', name='NDB_755', description='NDB_755', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCGTTTCGTATATGTCGATGTCCTTAGAC'), id='NDB_826', name='NDB_826', description='NDB_826', dbxrefs=[]),
  SeqRecord(seq=Seq('TACAAATACCGGTGGTTGTCACTTTCCGAA'), id='NDB_713', name='NDB_713', description='NDB_713', dbxrefs=[]),
  SeqRecord(seq=Seq('TCCGTGACGAGATCCCGGTCGGATATGTGA'), id='NDB_865', name='NDB_865', description='NDB_865', dbxrefs=[]),
  SeqRecord(seq=Seq('GAGCTTATGACTACCACGGCATCGATTTAG'), id='NDB_725', name='NDB_725', description='NDB_725', dbxrefs=[]),
  SeqRecord(seq=Seq('CGCGAAGATCGAACTCGTTGACTAAGCATT'), id='NDB_817', name='NDB_817', description='NDB_817', dbxrefs=[]),
  SeqRecord(seq=Seq('GTGGTAAACCTCAACTACTAAGCCGCGGTA'), id='NDB_710', name='NDB_710', description='NDB_710', dbxrefs=[]),
  SeqRecord(seq=Seq('CCAGTA

In [18]:
import pandas as pd
# generate the codebook
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
#codebook.add(['name', 'id'], axis=1)
codebook['id'] = list(region_2_readout_dict.keys())

In [25]:
codebook['name'] = [region_2_names[str(_r)] for _r in list(region_2_readout_dict.keys())]

In [26]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_1028,NDB_925,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910
0,1:3740000-3760000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1:6240000-6260000,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1:8740000-8760000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1:11240000-11260000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1:13740000-13760000,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,19:53740000-53760000,989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
984,19:56240000-56260000,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985,19:58740000-58760000,991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,19:61240000-61260000,992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1

In [38]:
for _gname, _bits in region_2_readout_dict.items():
    binary_code = []
    for _i in range(max_bit):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    print(_gname)
    codebook.loc[codebook['id']==_gname, codebook.columns[2:]] = binary_code
    #codebook.loc[codebook['name']==_gname,'id'] = int(_bits[0].split('u')[1])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
140
141
142
143
144
145
146
147
148
150
151
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
190
191
192
193
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
266
267
268
269
270
271
272
273
274
276
277
278
279
281
282
283
285
286
287
288
289
290
29

In [41]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_1028,NDB_925,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,19:53740000-53760000,989,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
984,19:56240000-56260000,990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
985,19:58740000-58760000,991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
986,19:61240000-61260000,992,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [42]:
chr_2_region_num = pickle.load(open(os.path.join(library_folder, 'chr_2_region_num.pkl'), 'rb'))

In [43]:
chr_2_region_num

{'1': 77,
 '2': 70,
 '3': 61,
 '4': 59,
 '5': 58,
 '6': 58,
 '7': 49,
 '8': 50,
 '9': 48,
 '10': 51,
 '11': 48,
 '12': 43,
 '13': 45,
 '14': 42,
 '15': 40,
 '16': 38,
 '17': 36,
 '18': 35,
 '19': 23,
 'X': 56,
 'Y': 1}

In [45]:
chr_2_region_names = pickle.load(open(os.path.join(library_folder, 'chr_2_region_names.pkl'), 'rb'))

In [46]:
chr_2_region_names

{'1': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77],
 '2': [80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  150,
  151],
 '3': [153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  16

In [47]:
# generate region_to_chr_id
region_2_chr = {}
region_2_chr_order = {}
for _chr, _regions in chr_2_region_names.items():
    for _i, _region in enumerate(_regions):
        region_2_chr[_region] = _chr
        region_2_chr_order[_region] = _i

In [48]:
codebook['chr'] = [region_2_chr[_g] for _g in codebook['id']]
codebook['chr_order'] = [region_2_chr_order[_g] for _g in codebook['id']]

In [54]:
codebook.sort_values('id', inplace=True)

In [56]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-1000_codebook.csv')

In [57]:
codebook.to_csv(save_filename,
                index=None)

# generate 500 gene codebook

In [79]:
codebook_500 = pd.DataFrame(columns=codebook.columns)

# extract 500 genes
sel_regions_500 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_500':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_500:
            sel_regions_500.append(_gene)
            
for _gene in sel_regions_500:
    codebook_500 = codebook_500.append(codebook.loc[codebook['id']==int(_gene)])
codebook_500.sort_values('id', inplace=True)
codebook_500

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,X:161240000-161260000,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
987,Y:1240000-1260000,1062,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Y,0


In [80]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-500_codebook.csv')

codebook_500.to_csv(save_filename, index=None)

# 250-1

In [81]:
codebook_250_1 = pd.DataFrame(columns=codebook.columns)

# extract 250_1 genes
sel_regions_250_1 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_1':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_1:
            sel_regions_250_1.append(_gene)
            
for _gene in sel_regions_250_1:
    codebook_250_1 = codebook_250_1.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_1.sort_values('id', inplace=True)
codebook_250_1

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,1:16240000-16260000,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
9,1:26240000-26260000,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9
13,1:36240000-36260000,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13
17,1:46240000-46260000,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,X:116240000-116260000,1039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,37
424,X:131240000-131260000,1045,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,41
428,X:141240000-141260000,1049,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,45
432,X:153740000-153760000,1054,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,49


In [82]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_1_codebook.csv')

codebook_250_1.to_csv(save_filename, index=None)

# 250-2

In [83]:
codebook_250_2 = pd.DataFrame(columns=codebook.columns)

# extract 250_2 genes
sel_regions_250_2 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_2':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_2:
            sel_regions_250_2.append(_gene)
            
for _gene in sel_regions_250_2:
    codebook_250_2 = codebook_250_2.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_2.sort_values('id', inplace=True)
codebook_250_2

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
7,1:21240000-21260000,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
11,1:31240000-31260000,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11
15,1:41240000-41260000,16,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,15
19,1:51240000-51260000,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,X:121240000-121260000,1041,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,39
426,X:136240000-136260000,1047,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,43
430,X:146240000-146260000,1051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,47
434,X:158740000-158760000,1056,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,51


In [84]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_2_codebook.csv')

codebook_250_2.to_csv(save_filename, index=None)